In [1]:
# ######################################################################
# Preparation
# ######################################################################

import os
import oqs
import ecdsa
import base64
import binascii
from pprint import pprint
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

# ######################################################################
# Check the library information
# ######################################################################

print("liboqs version:", oqs.oqs_version())
print("liboqs-python version:", oqs.oqs_python_version())
print("\nEnabled KEM mechanisms:")
kems = oqs.get_enabled_KEM_mechanisms()
pprint(kems, compact=True)
print()

liboqs version: 0.7.2
liboqs-python version: 0.7.2

Enabled KEM mechanisms:
['BIKE-L1', 'BIKE-L3', 'Classic-McEliece-348864', 'Classic-McEliece-348864f',
 'Classic-McEliece-460896', 'Classic-McEliece-460896f',
 'Classic-McEliece-6688128', 'Classic-McEliece-6688128f',
 'Classic-McEliece-6960119', 'Classic-McEliece-6960119f',
 'Classic-McEliece-8192128', 'Classic-McEliece-8192128f', 'HQC-128', 'HQC-192',
 'HQC-256', 'Kyber512', 'Kyber768', 'Kyber1024', 'Kyber512-90s', 'Kyber768-90s',
 'Kyber1024-90s', 'NTRU-HPS-2048-509', 'NTRU-HPS-2048-677', 'NTRU-HPS-4096-821',
 'NTRU-HPS-4096-1229', 'NTRU-HRSS-701', 'NTRU-HRSS-1373', 'ntrulpr653',
 'ntrulpr761', 'ntrulpr857', 'ntrulpr1277', 'sntrup653', 'sntrup761',
 'sntrup857', 'sntrup1277', 'LightSaber-KEM', 'Saber-KEM', 'FireSaber-KEM',
 'FrodoKEM-640-AES', 'FrodoKEM-640-SHAKE', 'FrodoKEM-976-AES',
 'FrodoKEM-976-SHAKE', 'FrodoKEM-1344-AES', 'FrodoKEM-1344-SHAKE']



In [14]:
# ######################################################################
# A Simple KEM Framework
# ######################################################################

# 0. Initialization
kem_algo = 'Kyber512'
sender = oqs.KeyEncapsulation(kem_algo)

# 1. Sender generates the key pair
public_key = sender.generate_keypair()
secret_key = sender.export_secret_key()

print('The following keys are displayed in part with utf-8 style.\n')
print('[Key pair generation] (by client)')
print('KEM public_key:', binascii.hexlify(public_key)[:10])
print('KEM secret_key:', binascii.hexlify(secret_key)[:10])
print()

# 2. Sender generates ciphertext and key 
ciphertext, key = sender.encap_secret(public_key)

print('[Encapsulation] (by sender)')
print('KEM ciphertext:', binascii.hexlify(ciphertext)[:10])
print('KEM key (sender):', binascii.hexlify(key)[:10])
print()

# 3. Receiver recovers the key by secret key and the ciphertext
receiver = oqs.KeyEncapsulation(kem_algo, secret_key)
key_receiver = receiver.decap_secret(ciphertext)

print('[Decapsulation] (by receiver)')
print('KEM key (receiver):', 
      binascii.hexlify(key_receiver)[:10])
print('\nThe key can be recovered by the ciphertext and secrete key.')
print('Be sure not to confuse secret key with secret.\n')

The following keys are displayed in part with utf-8 style.

[Key pair generation] (by client)
KEM public_key: b'4155b88656'
KEM secret_key: b'c609017109'

[Encapsulation] (by sender)
KEM ciphertext: b'ca130b9d38'
KEM key (sender): b'27d39a91d2'

[Decapsulation] (by receiver)
KEM key (receiver): b'27d39a91d2'

The key can be recovered by the ciphertext and secrete key.
Be sure not to confuse secret key with secret.



In [2]:
# ######################################################################
# Helper Function
# ######################################################################
def key_transform(key, salt):
    """
    This function transforms an KEM key to the format
    of 32 url-safe base64-encoded bytes.
    
    Input: 
        key (bytes in binary format): the key from KEM
        salt (bytes): a string generated by os.urandom(16)
    Return: 
        key_ (bytes in base64 format): the key used for SKE
    
    Source: cryptography.io/en/latest/
            fernet/#using-passwords-with-fernet
    """
    password = binascii.hexlify(key)
    kdf = PBKDF2HMAC(algorithm=hashes.SHA256(),
                     length=32,
                     salt=salt,
                     iterations=480000)
    key_ = base64.urlsafe_b64encode(kdf.derive(password))
    
    return key_

In [8]:
# ######################################################################
# KEM for Message Encryption
# ######################################################################

# ======================================================================
# 0. Initialization
# ======================================================================
kem_algo = 'Kyber512'
sender = oqs.KeyEncapsulation(kem_algo)

# 1. Sender generates the key pair
public_key = sender.generate_keypair()
secret_key = sender.export_secret_key()

print('The following keys are displayed in part with utf-8 style.\n')
print('========= Key pair generation (by sender) =========')
print('KEM public key:', binascii.hexlify(public_key)[:10])
print('KEM secret key:', binascii.hexlify(secret_key)[:10])
print()


# ======================================================================
# 2. Sender Encryption
# ======================================================================
# 2.1. Sender generates ciphertext and key 
ciphertext, key = sender.encap_secret(public_key)

print('========= Encryption (by sender) =========')
print('------ Step 1: Encapsulation ------')
print('KEM ciphertext:', binascii.hexlify(ciphertext)[:10])
print('KEM key (sender):', binascii.hexlify(key)[:10])
print()

# 2.2. Encrypt the private key on web3 (i.e., message)
# 2.2.1. Encrypt the private key used on blockchain in utf-8 format
message = base64.b64encode(os.urandom(32)).decode('utf-8')
message = '5906cbe7c9939c405a29b54226a26017dccee89e395d219b7f63e2dc8ed3b9ce'
print('------ Step 2: Symmetric Encryption ------')
print(f'Original web3 private key:\n{message}.\n')

# 2.2.2. Transform the key to a symmetric key (p.s. info may lose here)
salt = os.urandom(16)
key_ = key_transform(key, salt)

# 2.2.3. Encrypt the private key used on blockchain by the transformed key
fernet = Fernet(key_)
private_key_web3_encrypted = fernet.encrypt(message.encode())
print(f'Encrypted web3 private key: '
      f'(symmetrically encrypted using the KEM key)'
      f'\n{private_key_web3_encrypted}.\n')
print(f'Outputs for the receiver: \n'
      '- encrypted web3 private key\n'
      '- ciphertext\n'
      '- secret key\n'
      '- salt (for key transformation)\n')
print('Our clients will only hold the encrypted web3 private key.\n'
      'They can request the other three from us if needed under 2FA.\n')

# ======================================================================
# 3. Receiver Decryption
# ======================================================================
# 3.1. Receiver recovers the key by secret key and the ciphertext
receiver = oqs.KeyEncapsulation(kem_algo, secret_key)
key_receiver = receiver.decap_secret(ciphertext)

print('========= Decryption (by receiver) =========')
print('------ Step 1: Decapsulation ------')
print('Decapsulated KEM key:', 
      binascii.hexlify(key_receiver)[:10], '(decapsulated using ciphertext and secret key)\n')

# 3.2. Receiver decrypt the message by the key
key_receiver_ = key_transform(key_receiver, salt)
fernet_receiver = Fernet(key_receiver_)
private_key_web3_decrypted = fernet_receiver.decrypt(private_key_web3_encrypted).decode()
print('------ Step 2: Symmetric Decryption ------')
print(f'The receiver then decrypt the web3 private key '
      f'(by the decapsulated KEM key and salt):\n{private_key_web3_decrypted}.')
print()


The following keys are displayed in part with utf-8 style.

========= Key pair generation (by sender) =========
KEM public key: b'bd9c1db016'
KEM secret key: b'0b1367ff06'

========= Encryption (by sender) =========
------ Step 1: Encapsulation ------
KEM ciphertext: b'de857d69a0'
KEM key (sender): b'fede916172'

------ Step 2: Symmetric Encryption ------
Original web3 private key:
5906cbe7c9939c405a29b54226a26017dccee89e395d219b7f63e2dc8ed3b9ce.

Encrypted web3 private key: (symmetrically encrypted using the KEM key)
b'gAAAAABjE2-OTl6s-9YP8KSLn-1dJ79_wJVR1Nd5tJrVrvKKLIhu-8gJ1yGVzDxxGaJ70rTvioEqEfT-P4GkjntycyWBJKazqHosWIKpv_gv0RV8OhIaciWRiZrFx2Z2PcSjs2qpCU0ZXaxF54VLQ6bIQARNVn3FTpLm7d57Sp8_G2kgJANHEmw='.

Outputs for the receiver: 
- encrypted web3 private key
- ciphertext
- secret key
- salt (for key transformation)

Our clients will only hold the encrypted web3 private key.
They can request the other three from us if needed under 2FA.

========= Decryption (by receiver) =========
----